# Deep Learning and Neural Networks
Deep Learning is a subset of machine learning algorithms. It is based on neural networks and is capable of learning complex patterns within the data. It's advantage over simple machine learning methods lies in the fact that it can be built by stacking a number of layers of neurons to discover heirarchical patterns of featurese within the data. Thus, it should be able to extract more complex features not otherwise detected by simpler machine learning algorithms

<img src="https://developer.ibm.com/developer/default/articles/an-introduction-to-deep-learning/images/AI-ML-DL.png" width=300 height=300 />

Image source: [IBM Developer](https://developer.ibm.com/articles/an-introduction-to-deep-learning/)

## Neural Networks
The name "Neural Networks" comes from the structure of neurons in our brains, where multiple neurons connect to each other in layers to perform their respective functions. Although the similarity is vague, the name still catches on. To underestand this concept, let's start with explaining what is a neuron.

#### The Neuron
The basic building block of neural network is the neuron (sometimes called perceptron). It can be thought of as simple functions that take inputs and convert them to outputs. This conversion function is known as the activation function. In each neuron, the following steps take place:

* Take the from each variable in the input layer and multiply it by a factor (weight)
* Sum the product of inputs and weights and add a bias term (b)
* Apply a function (activation function) to the final sum

Let's assume we have a data for predicting the metastasis of a primary hepatic tumor. We collected four variables to be used for our task. These are the alpha fetoprotein (AFP), number of risk genes, alanine transaminase (ALT)
<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/dl_neuron.png" width=1000 height=320 />

There is a number of activation functions that are commonly used. These include the sigmoid function and ReLU function.

#### Building Neural Networks
The most basic form of neural networks consists of three layers: the input layer, a hidden layer and an output layer. In the hidden layer, neurons are arranged so that they receive input from the input layer and apply their activation functions on the input. The results are then passed to the output layer to give the final output.